In [1]:
class Member:
    def __init__(self,login_id,name,password):
        self.login_id=login_id
        self.name=name
        self.password=password
    def __str__(self):
        return(f"login_id: {login_id}, name:{self.name},"
               f"password:{self.password}")
    def to_dict(self):
        return{"login_id":self.login_id,"name":self.name, 
               "password":self.password}

In [18]:
def create_table():
    cursor=conn.cursor()
    sql_board="create table board1(nu number(10),title varchar2(1000),login_id varchar2(20),uptime date,coun number(10),name varchar2(20),content varchar2(1000))"
    cursor.execute(sql_board)
    sql_login="create table login(login_id varchar2(20),name varchar2(20),password varchar2(10))"
    cursor.execute(sql_login)
    sql_boardinfo="insert into board1 values(1,'처음게시물','id',sysdate,3,'이름','내용')"
    cursor.execute(sql_boardinfo)
    conn.commit()

In [4]:
def login_info():
    cursor=conn.cursor()
    print("*회원가입*")
    login_id=input("아이디를 입력해 주세요:")
    name=input("이름을 입력해 주세요:")
    password=input("비밀번호를 입력해 주세요:")
    sql= "insert into login values(:login_id,:name,:password)"
    member=Member(login_id,name,password)
    cursor.execute(sql,member.to_dict())
    print(f"{name}({login_id}) 회원님 가입을 축하합니다.")

In [5]:
def login():
    print("*로그인*")
    login_id=input("아이디:")
    password=input("비밀번호:")
    cursor=conn.cursor()
    sql='select * from login where login_id=:login_id and password=:password'
    cursor.execute(sql,(login_id,password))
    result=cursor.fetchall()
    member=Member(result[0][0],result[0][1],result[0][2])
    if member.login_id==login_id and member.password==password:
        print(f"{member.name}({login_id}) 회원님이 로그인 하였습니다.")
    session=[member.name,login_id]
    return session

In [6]:
def check_number():
    cursor=conn.cursor()
    sql='select max(nu) from board1'
    cursor.execute(sql)
    result=cursor.fetchone()[0]
    return int(result)+1
        

In [7]:
def write_board(session):
    nu=check_number()
    print("번호:{}".format(nu))
    print("작성자:{}({})".format(session[0],session[1]))
    title=input("제목:")
    content=input("내용:")
    print("글이 입력되었습니다.")
    print("조회수:3")
    print("글목록:l, 로그아웃:o")
    cursor=conn.cursor()
    sql='insert into board1 values(:nu,:title,:login_id,sysdate,3,:name,:content)'
    cursor.execute(sql,(nu,title,session[0],session[1],content))
    write=input("선택>")
    return write.lower() 

In [8]:
def board_info(board):
    cursor=conn.cursor()
    sql= "select title,content from board1 where nu=:board"
    cursor.execute(sql,(board,))
    result=cursor.fetchone()
    print(result)
    print("*상세보기*")
    print('제목:{}'.format(result[0]))
    print('내용:{}'.format(result[1]))
    print("글목록:l, 글삭제:d, 로그아웃:o")
    back=input("선택>")
    return back.lower()

In [9]:
def delete_board(board):
    cursor=conn.cursor()
    sql='delete from board1 where nu=:board'
    cursor.execute(sql,(board,))
    print("삭제되었습니다.")

In [10]:
def select_menu():
    print("*메뉴*","1.회원가입","2.로그인","3.종료",sep="\n")
    menu=input("선택:")
    return int(menu)

In [11]:
def main():
    while True:
        menu=select_menu()
        if menu==1:
            login_info()
            conn.commit()
        elif menu==2:
            session=login()
            board=select_board()
            if board=='o':
                print("로그아웃되었습니다.")
                break
            elif board=='w':
                write=write_board(session)
                conn.commit()
                if write=='l':
                    select_board()
                elif write=='o':
                    break
            else:
                back=board_info(board)
                if back=='l':
                    select_board()
                elif back=='d':
                    delete_board(board)
                    conn.commit()
                    select_board()
                elif back=='o':
                    print("로그아웃되었습니다.")
                    break  
        elif menu==3:
            print("종료되었습니다.")
            break

In [12]:
def select_board():
    print("*글목록*")
    print('-'*6+'+'+'-'*15+'+'+'-'*10+'+'+'-'*10)
    print('{:<5}{:<14}{:<8}{:<9}{:<9}'.format("번호","제목","작성자","날짜","조회수"))
    cursor=conn.cursor()
    sql="select nu,title,login_id,to_char(uptime,'yyyy-mm-dd'),coun from board1 order by nu"
    cursor.execute(sql)
    result=cursor.fetchall()
    for board in result:
        print('{:<7}{:<16}{:<11}{:<15}{:<10}'.format(board[0],board[1],board[2],board[3],board[4]))
    print('-'*6+'+'+'-'*15+'+'+'-'*10+'+'+'-'*10)
    print("글쓰기:w,읽기:번호,로그아웃:o")
    select=input("선택>")
    return select if select.lower()=='o' or select.lower()=='w' else int(select)
    

In [13]:
import cx_Oracle
oracle_dsn=cx_Oracle.makedsn(host="localhost", port=1521,sid="xe")
conn=cx_Oracle.connect(dsn=oracle_dsn,user="hr",password="hr")

In [20]:
# 초기 테이블 생성하는 함수 최초 1회만 실행!
create_table()

DatabaseError: ORA-00955: 기존의 객체가 이름을 사용하고 있습니다.

In [21]:
main()

*메뉴*
1.회원가입
2.로그인
3.종료
선택:2
*로그인*
아이디:rlaehddud
비밀번호:1
김동영(rlaehddud) 회원님이 로그인 하였습니다.
*글목록*
------+---------------+----------+----------
번호   제목            작성자     날짜       조회수      
1      처음게시물           id         2021-11-23     3         
------+---------------+----------+----------
글쓰기:w,읽기:번호,로그아웃:o
선택>w
번호:2
작성자:김동영(rlaehddud)
제목:안녕
내용:두번째
글이 입력되었습니다.
조회수:3
글목록:l, 로그아웃:o
선택>L
*글목록*
------+---------------+----------+----------
번호   제목            작성자     날짜       조회수      
1      처음게시물           id         2021-11-23     3         
2      안녕              김동영        2021-11-23     3         
------+---------------+----------+----------
글쓰기:w,읽기:번호,로그아웃:o
선택>o
*메뉴*
1.회원가입
2.로그인
3.종료
선택:3
종료되었습니다.
